In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [82]:
## Basic tools ##

def gcd(a:int,b:int)->int:
    a = abs(a)
    b = abs(b)
    if min(a,b)==0:
        return max(a,b)
    while b% a != 0:
        r = b % a
        b = a
        a = r
    return a

def beta_mat(abc:tuple[int],xy:tuple[int])->list[list[int]]:
    a,b,c = abc
    x,y = xy
    return [[y-b*x,a*x],[-c*x,y]]

def hall_multiplier(l:int,m:int)->int:
    if min(l,m)< 1:
        return 'Not defined'
    l0 = gcd(l,m)
    l1 = m//l0
    multiplier = 1
    while gcd(l0,l1)>1:
        g = gcd(l0,l1)
        l0*= g
        l1 = l1//g
        multiplier*=g
    return multiplier

def kernel_gen_cyc(abc:tuple[int], xy:tuple[int])->dict:
    m = beta_mat(abc,xy)
    m00 = m[0][0]
    m01 = m[0][1]
    m10 = m[1][0]
    m11 = m[1][1]
    l = m00*m11 - m01*m10
    l0 = gcd(gcd(m00,l),gcd(m01,l))
    l1 = gcd(gcd(m10,l),gcd(m11,l))
    if gcd(l0,l1)>1:
        return 'Check gcds'
    n0 = hall_multiplier(l0,l)
    # Multiplying u by n0 gives us a point that generates a subgroup
    # of index l0c; the order of the point is l/l0c
    l0c = n0*l0
    # We need a point of order l0c, and index l/l0c
    # We already have a point of order l/l1
    n1 = l//(l0c*l1)
    # v already has order l/l1
    # (l/l1) *v = 0, so (l/l1)/(l0c) will have order l0c
    xg = -(n0*m01+n1*m11)%l
    yg = (n0*m00+n1*m10)%l
    return {(xg,yg):l}


def red_bf(abc:tuple)->bool:
    a,b,c = abc
    if abs(b)< a and a < c:
        return True
    elif b == a and a < c:
        return True
    elif c == a and b >= 0 and b <= a:
        return True
    return False

# Input: a discriminant d
# Output: The list of reduced binary quadratic forms with discriminant d

def get_cl_reps(d:int)->list:
    reps_found = []
    # First we check that d is indeed a discriminant;
    # if it isn't, we simply return an empty list because there are no associated lattices
    if d % 4 > 1 or d >= 0:
        return reps_found
    b = d % 4
    while 3*b *b <= abs(d):
        num = (b*b-d)//4
        a = b
        while a *a <= num:
            if a == 0:
                a+=1
            if num % a == 0:
                c = num//a
                if red_bf((a,b,c)):
                    reps_found.append((a,b,c))
                if b!= 0 and red_bf((a,-b,c)):
                    reps_found.append((a,-b,c))
            a+=1
        b+=2
    return reps_found

def dot(v,w):
    return sum([v[i]*w[i] for i in range(min(len(v),len(w)))])

def det(m):
    return m[0][0]*m[1][1]-m[0][1]*m[1][0]

def matvec(m,v):
    return [dot(r,v) for r in m]

In [83]:
xys = [(x*m,y*m) for x in range(1,10) for y in range(-10,10) for m in range(1,6) if y!=0]
xys_prim = [xy for xy in xys if gcd(xy[0],xy[1])==1]
discs = [-d for d in range(1,204) if (-d)%4 <2]
abcs = []
for d in discs:
    abcs+=get_cl_reps(d)

def make_prim(abc):
    a,b,c = abc
    g = gcd(gcd(a,b),c)
    if g == 0:
        return abc
    return a//g,b//g,c//g

abcs_prim = list({make_prim(abc) for abc in abcs})

In [85]:
def test_cyc(abc:tuple[int],xy:tuple[int]):
    failedtests = []
    m = beta_mat(abc,xy)
    l = det(m)
    gdic = kernel_gen_cyc(abc,xy)
    v = [v for v in gdic][0]
    mv = matvec(m,v)
    x,y = v
    mx = mv[0]
    my = mv[1]
    if min(mx%l,my%l)>0:
        failedtests.append('Not in kernel')
    if gcd(gcd(x,y),l)>1:
        failedtests.append('Wrong order')
    return failedtests
        


In [88]:
passed = []
failed = {}

for abc in abcs_prim:
    for xy in xys_prim:
        tax = test_cyc(abc,xy)
        inp = (abc,xy)
        if len(tax)>0:
            failed[inp]=tax
        else:
            passed.append(inp)

In [89]:
len(failed)

0

In [1]:
import os
os.chdir('../../py/lattices')
from pointgroups import *

In [2]:
smallprimes = [2,3]
nums = [n for n in range(5,100,2) if n % 3 !=0]
while len(nums)>0:
    p = nums.pop(0)
    smallprimes.append(p)
    nums = [n for n in nums if n % p != 0]

In [5]:
pointgrouptest = {(xy,abc):kernel_generators(xy,abc)
                  for xy in xys
                  for abc in abcs_prim}

In [24]:
def checksize(grp):
    basis = [v for v in grp]
    n = max(grp.values())
    grps =[]
    failedtests = []
    for v in basis:
        x,y = v
        grv = {((a*x)% n,(a*y)%n) for a in range(n)}
        if len(grv)!= grp[v]:
            failedtests.append(str(v)+" has wrong order")
        grps.append(grv)
    if len(grps)>1:
        g1 = grps[0]
        g2 = grps[1]
        if len(g1.intersection(g2))>1:
            failedtests.append('Generators not independent')
    return failedtests


In [59]:
def check_kernel(input,output):
    failedtests = []
    xy, abc = input
    n = 1
    n0 = 1
    for v in output:
        n*=output[v]
        n0 = max(n0,output[v])
    a,b,c = abc
    g = gcd(a,gcd(b,c))
    a,b,c = a//g,b//g,c//g
    x,y = xy
    mat = [[y-b*x,a*x],[-c*x,y]]
    det = mat[0][0]*mat[1][1]-mat[1][0]*mat[0][1]
    if det!= n:
        failedtests.append('Expected size was incorrect')
    for v in output:
        xv, yv = v
        mxv = (mat[0][0]*yv+mat[0][1]*xv)% n0
        myv = (mat[1][0]*xv+mat[1][1]*yv)% n0
        if max(mxv,myv)>0:
            failedtests.append(str(v)+' not in kernel')
    return failedtests

badoutputs2 = {pair:check_kernel(pair,pointgrouptest[pair])
               for pair in pointgrouptest}
badoutputs2 = {pair:badoutputs2[pair] for pair in badoutputs2 
               if len(badoutputs2[pair])>0}




In [60]:
len(badoutputs2)

275281

In [65]:
def axby2(ab:tuple[int])->tuple[int]:
    a, b = ab
    x0, y0, x1, y1 = 0, 1, 1, 0
    while a != 0:
        r = b % a
        q = b // a
        x2 = q*x1 + x0
        x0 = x1
        x1 = x2
        y2 = q*y1 + y0
        y0 = y1
        y1 = y2
        b = a
        a = r
    return x0,y0,x1,y1

def beta_mat(abc:tuple[int],xy:tuple[int])->list[list[int]]:
    a,b,c = abc
    x,y = xy
    return [[y-b*x,a*x],[-c*x,y]]

In [64]:
beta_mat(abcs_prim[3],xys_prim[4])

[[-7, 1], [-46, -6]]

In [69]:
x0,y0,x1,y1=axby2([(r[0],r[1]) for r in beta_mat(abcs_prim[3],xys_prim[4])][0])

In [70]:
x0,y0,x1,y1

(0, 1, -1, 7)

In [77]:
matvec(beta_mat(abcs_prim[3],xys_prim[4]),[1,7])

[0, -88]

In [75]:
matvec([[1,10],[1024,193]],[-1,1])

[9, -831]

In [80]:
det(beta_mat(abcs_prim[3],xys_prim[4]))

88

In [25]:
badoutputs1 = {pair:checksize(pointgrouptest[pair]) for pair in pointgrouptest}
badoutputs1 = {pair:badoutputs1[pair] for pair in badoutputs1 if len(badoutputs1[pair])>0}

In [26]:
len(badoutputs1)

0

In [7]:
def check_pgoutput(input,output):
    failed_tests = []
    xy, abc = input
    x,y = xy
    a,b,c = abc
    lm2 = a*c*(x*x) + y*(y-b*x)
    # We should have only one generator if x,y are rel prime
    m = gcd(x,y)
    expectedgens = 2-int(m==1)
    if expectedgens != len(output):
        failed_tests.append('Wrong number of generators')        
    if lm2 % (m*m)!= 0:
        failed_tests.append('Check l, m')
    l = lm2//(m*m)
    # The cardinality should be the product of the keys
    vecs = [v for v in output]
    vecs.sort(key = lambda x:output[x])
    if len(output)==1:
        v = vecs[0]
        if output[v]!= l:
            failed_tests.append('Generator has wrong expected order')
        else:
            x,y = v
            if gcd(gcd(x,y),l)>1:
                failed_tests.append('Generator does not have expected order')
    else:
        v0 = vecs[0]
        v1 = vecs[1]
        x0,y0 = v0
        x1,y1 = v1
        if output[v0]*output[v1]!= lm2:
            failed_tests.append('Generators have wrong expected order')
        if gcd(gcd(x0,y0),l*m) != l:
            failed_tests.append('Small gen has wrong expected order')
        if gcd(gcd(x1,y1),l*m) != l*m:
            failed_tests.append('Big gen does not have expected order')
    return failed_tests





In [14]:
badinputs = [i for i in tests]

In [9]:
tests = {inp:check_pgoutput(inp,pointgrouptest[inp]) for inp in pointgrouptest}

In [17]:
failedtests = []
for t in tests:
    failedtests+=tests[t]
failedtests = list(set(failedtests))

In [20]:
failedtests

['Big gen does not have expected order', 'Wrong number of generators']

In [12]:
len(tests)

241248

In [11]:
[pointgrouptest[s] for s in tests]

[{2: (0, 1)},
 {3: (0, 1)},
 {4: (0, 1)},
 {5: (0, 1)},
 {6: (0, 1)},
 {8: (0, 1)},
 {10: (0, 1)},
 {9: (0, 1)},
 {12: (0, 1)},
 {15: (0, 1)},
 {16: (0, 1)},
 {20: (0, 1)},
 {25: (0, 1)},
 {18: (0, 1)},
 {24: (0, 1)},
 {30: (0, 1)},
 {7: (0, 1)},
 {14: (0, 1)},
 {21: (0, 1)},
 {28: (0, 1)},
 {35: (0, 1)},
 {32: (0, 1)},
 {40: (0, 1)},
 {27: (0, 1)},
 {36: (0, 1)},
 {45: (0, 1)}]

## Overview

Suppose we have a lattice $\Lambda$ and a complex number $\alpha$ with the property that $\alpha \Lambda \subset \Lambda$ - we use this to define an elliptic curve endomorphism $z + \Lambda \mapsto \alpha z + \Lambda$.
Our goal is to find a basis for the kernel of this endomorphism.

Here is the precise set-up:

* To describe the lattice, we just need to specify $\tau$. 
We do this using the minimal polynomial of $\tau$ over the rationals: this is a quadratic of the form $ax^2 + bx + c$, with $a,b,c$ integers. If we know the coefficients $(a,b,c)$ of the quadratic, then $\tau$ is the unique root of $ax^2 + bx + c = 0$ in the upper half plane and $\Lambda$ is the lattice with basis $\tau, 1$.

* To describe $\alpha$, we also use a minimal polynomial. Unlike $\tau$, $\alpha$ is necessarily an algebraic integer, so the minimal polynomial of $\alpha$ has the form $x^2 - t x + n$, where $t$ is the trace of $\alpha$ and $n$ the norm of $\alpha$. To describe $\alpha$, we will specify the trace and norm. If we are given a trace and a norm, the $\alpha$ they represent is the root of $x^2 - tx + n = 0$ in the upper half plane.

* We have to choose both $\tau, \alpha$ but the choices must be compatible - that is, $\alpha$ should be an endomorphism of the lattice generated by $\tau, 1$. Fortunately, this can easily be checked using the minimal polynomials: we just have to make sure that $t^2-4n = m^2 (b^2-4ac)$ - i.e. the discriminants should have the same squarefree part, and the discriminant of the lattice should divide the discriminant of the endomorphism.

Important notes: Be careful with the sign of the trace - when we use $a,b,c$ the quadratic form is $ax^2 + bx + c$, when we use $t,n$ the quadratic is $x^2 - tx + n$.

Also, remember that $\alpha$ is always assumed to be an element of the upper half plane - we will also need to use the conjugate of $\alpha$, and the only way to tell them apart is by remembering that the sign of the imaginary part of $\alpha$ is always chosen to match the sign of the imaginary part of $\tau$.


Now, given $(a,b,c), (t,n)$, we will look for elements in the usual fundamental domain that can be used to generate the kernel of the associated endomorphism.

### 

We're going to compute...

### Number Theory Code

The following code is needed for the function 'pointgroup_roi'.

This function will take as input the following:
* A pair $(a,p)$, that represents the equivalence class of curves mod $p$ with trace of Frobenius equal to $a$.
* An integer $n$.


It will return an array $[[x_0,y_0],[x_1,y_2]..]$,
where the $[x_i,y_i]$ are points in the fundamental domain of $\mathbb{C}/\Lambda$,
where the lattice $\Lambda$ is the ring of integers of $\mathbb{Q}(\sqrt{a^2-4p})$.
* The ring of integers appears in every equivalence class of lattices.
* If the equivalence class contains exactly one element, then that element must be the ring of integers.

So, if we only want to make a single picture for a given equivalence class, or we're making pictures of singleton equivalence classes, then the following code is enough to obtain the point group. 

In [2]:
def gcd(a,b):
    a = abs(a)
    b = abs(b)
    if min(a,b)==0:
        return max(a,b)
    while b% a !=0:
        r = b% a
        b = a
        a = r
    return a


def gcd_list(l):
    d = 0
    for a in l:
        d = gcd(a,d)
    return d

def primefac(n):
    if n == 0:
        return {0:1}
    pf = {}
    if n < 0:
        pf[-1]=1
        n = abs(n)
    if n % 2 == 0:
        pf[2] = 0
        while n % 2 == 0:
            n = n//2
            pf[2]+=1
    if n % 3 == 0:
        pf[3] = 0
        while n % 3 == 0:
            n = n//3
            pf[3]+=1
    d = 5
    e = -1
    while d*d <= n:
        if n % d == 0:
            pf[d] = 0
            while n % d == 0:
                n = n//d
                pf[d]+=1
        d+=3+e
        e*=-1
    if n > 1:
        pf[n]=1
    return pf

def pf_to_int(pf):
    n = 1
    for p in pf:
        n*= p**pf[p]
    return n


def quad_gcd(a,b):
    pf1 = primefac(a)
    pf2 = primefac(b)
    if a == 0:
        pfgcd = {p:pf2[p]//2 for p in pf2}
        return pf_to_int(pfgcd)
    pfgcd = {}
    for p in pf1:
        if p in pf2:
            pfgcd[p] = min(pf1[p],pf2[p]//2)
    return pf_to_int(pfgcd)

def discfac(d):
    pfd = primefac(d)
    rt = pf_to_int({p:pfd[p]//2 for p in pfd})
    fd = d//(rt*rt)
    if fd % 4 > 1:
        fd*=4
        rt = rt//2
    return fd, rt

## Matrix computations
def matrixgenfromd(d):
    return np.matrix([[0,d//4],[1,-(d%4)]])

def frobmatrixfromminpoly(ap):
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    one = np.matrix([[1,0],[0,1]])
    tau = matrixgenfromd(d0)
    if d0 % 4 == 0:
        return (a//2)*one+m*tau
    else:
        return ((a+m)//2) *one + m*tau

def gcdmat(mat):
    arr = np.array(mat).reshape(1,-1)[0]
    return gcd_list([int(a) for a in arr])

def matdet(m):
    m = np.array(m)
    return m[0,0]*m[1,1]-m[0,1]*m[1,0]

def mattr(m):
    m = np.array(m)
    return m[0,0]+m[1,1]

def mwexp_frommat(m):
    t = mattr(m)
    d = matdet(m)
    return abs(quad_gcd(t,d))

def matconj(m):
    return m - mattr(m)*np.matrix([[1,0],[0,1]])

def cx_to_arr(z)->np.array:
    return np.array([z.real,z.imag])

def pointgroup_roi(ap,n):
    frob = frobmatrixfromminpoly(ap)
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    frobpowerminusone= np.linalg.matrix_power(frob,n)-np.matrix([[1,0],[0,1]])
    mw2exp = mwexp_frommat(frobpowerminusone)
    mwcycmat = frobpowerminusone//mw2exp
    cycorder = matdet(mwcycmat)
    if cycorder == 1:
        one_arr = np.array([1,0])
        taumat = matrixgenfromd(d0)
        tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
        tau_cx = np.roots(tau_cp_coefs)[0]
        tau_arr = cx_to_arr(tau_cx)
        return np.array([a*one_arr+b*tau_arr
                         for a in range(mw2exp)
                         for b in range(mw2exp)])/mw2exp
    biggenorder = mw2exp*cycorder
    generator = np.array(matconj(frobpowerminusone))
    x = generator[0,0]%biggenorder
    y = generator[1,0]%biggenorder
    xygcd = gcd(x,y)
    xg,yg = (x//xygcd,y/xygcd)
    gen1intarr = np.array([xg,yg])
    gen2aintarr = np.array([cycorder,0])
    gen2bintarr = np.array([0,cycorder])
    xyints_all = {tuple((a1*gen1intarr+a2*gen2aintarr+a3*gen2bintarr)% biggenorder)
                  for a1 in range(biggenorder)
                  for a2 in range(mw2exp)
                  for a3 in range(mw2exp)}
    one_arr = np.array([1,0])
    taumat = matrixgenfromd(d0)
    tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
    tau_cx = np.roots(tau_cp_coefs)[0]
    tau_arr = cx_to_arr(tau_cx)
    points = np.array([xy[0]*one_arr+xy[1]*tau_arr for xy in xyints_all])/biggenorder
    return points

In [3]:

def axby(ab:tuple[int])->tuple[int]:
    a, b = ab
    x0, y0, x1, y1 = 0, 1, 1, 0
    while a != 0:
        r = b % a
        q = b // a
        x2 = q*x1 + x0
        x0 = x1
        x1 = x2
        y2 = q*y1 + y0
        y0 = y1
        y1 = y2
        b = a
        a = r
    return x0,y0

['pointgroups.py']

In [21]:
def sqrtFind(n):
    r = 1
    while r**2 < n:
        b = 2
        while (r+b)**2 < n:
            b*=2
        r+= (b//2)
    return r

In [26]:
def sqrtFind(n):
    if n <1:
        return 0
    r = 1
    while r**2 < n:
        b = 2
        while (r+b)**2 < n:
            b*=2
        r+= (b//2)
    return r

def getxy(bqf:tuple[int],mp:tuple[int]):
    a,b,c = bqf
    t,n = mp
    d0 = b*b-4*a*c
    d1 = t*t-4*n
    if d1 % d0 != 0:
        return 'Check discriminants'
    m = sqrtFind(d1//d0)
    if m*m*d0 != d1:
        return 'Check discriminants 2'
    x = (b*m+t)//2
    y = a*m
    return x,y

def getbasis_cyc(bqf:tuple[int],mp:tuple[int]):
    a,b,c = bqf
    t,n = mp
    d0 = b*b-4*a*c
    d1 = t*t-4*n
    if d1 % d0 != 0:
        return 'Check discriminants'
    m = sqrtFind(d1//d0)
    if m*m*d0 != d1:
        return 'Check discriminants 2'
    x = (b*m+t)//2
    y = a*m
    return (t-x)%n,(-y)%n
    

In [25]:
getxy((2,1,3),(12,59))

(-5, 4)

## Examples

I'm going to use the table of precomputed elliptic curve data to find pairs $(a,p)$ that have only one element in their equivalence class.

In [3]:
ellcurvedata = pd.read_pickle('../dataframes/elliptic_curve_counts.pk')

In [19]:
ellcurvedata

,TraceFrob,Prime,Discriminant,DiscriminantFac,jInvariants,degree
ap,,,,,,
"(0, 5)",0,5,-20,"(-20, 1)",[0],2
"(1, 5)",1,5,-19,"(-19, 1)",[4],1
"(2, 5)",2,5,-16,"(-4, 2)","[1, 3]",2
"(3, 5)",3,5,-11,"(-11, 1)",[2],1
"(4, 5)",4,5,-4,"(-4, 1)",[3],1
...,...,...,...,...,...,...
"(251, 16381)",251,16381,-2523,"(-3, 29)","[6190, 6252, 6149, 8256, 6959, 1809, 10431, 10...",11
"(252, 16381)",252,16381,-2020,"(-2020, 1)","[7524, 11748, 7622, 9234, 16286, 6173, 2645, 7...",8
"(253, 16381)",253,16381,-1515,"(-1515, 1)","[7110, 8942, 10621, 1981, 9728, 3817, 4713, 58...",12


The relevant discriminants are stored in the list 'goodds'.

For each of the relevant discriminants, the dictionary goodpairs stores a list of all pairs $(a,p)$ with that discriminant, with $p < 2^{14}$ and $a$ always nonnegative.

In [4]:
goodpairs = {-7:list(ellcurvedata.loc[ellcurvedata.Discriminant == -28].index)}
goodds = [-19,  -11,   -4,   -3,  -43,   -8,  -67, -163]
for d in goodds:
    goodpairs[d] = list(ellcurvedata.loc[ellcurvedata.Discriminant == d].index)
for d in goodpairs:
    goodpairs[d].sort(key= lambda x:x[1])

For each equivalence class, we pick out the pair $(a,p)$ with smallest $p>3$ that appears in the list. *I'm taking the second smallest for $p = 7$ because the smallest is $(0,7)$ and I don't want to deal with $a=0$ yet.*

In [5]:
smallest_examples = [goodpairs[d][0+int(d==-7)] for d in goodpairs]
smallest_examples_twist = [(-ap[0],ap[1]) for ap in smallest_examples]

In [6]:
smallest_examples

[(4, 11), (1, 5), (3, 5), (4, 5), (5, 7), (1, 11), (6, 11), (1, 17), (1, 41)]

We want to keep the number of points bounded above by ~15K (I'm going to use $2^{14}$ as the actual upper bound). The number of points on an elliptic curve over $\mathbb{F}_{p^n}$ is approximately $p^n$, so we will determine largest powers $n$ such that $p^n < 2^{14}$ for the primes in the smallest examples list.

In [7]:
primeexpbounds = {p[1]:[1,p[1]] for p in smallest_examples}
for p in primeexpbounds:
    while primeexpbounds[p][1]*p<2**14:
        primeexpbounds[p][0]+=1
        primeexpbounds[p][1]*=p

In [8]:
smallest_examples_pointgroups = {ap:[pointgroup_roi(ap,n) 
                                     for n in range(1,1+primeexpbounds[ap[1]][0])] 
                                     for ap in smallest_examples+smallest_examples_twist}

In [18]:
max(smallest_examples_pointgroups[(-5,7)][3][::,0])

0.9848675914249685

In [19]:
import os

In [20]:
for d in goodpairs:
    os.mkdir(str(d))
    
for ap in smallest_examples_pointgroups:
    a, p = ap
    d = discfac(a*a-4*p)[0]
    imagepath = str(d)+'/'+str(a)+'_'+str(p)+'/'
    os.mkdir(imagepath)
    for n, gr in enumerate(smallest_examples_pointgroups[ap]):
        file = open(imagepath+str(n+1)+'.txt','a')
        for i in range((len(gr)//3)+1):
            stri = ''
            for vec in gr[3*i:3*(i+1)]:
                stri += '['+str(vec[0])[:10]+','+str(vec[1])[:10]+'],'
            stri+='\n'
            file.write(stri)
        file.close()


In [73]:
n = 5
gr = pointgroup_roi((-5,7),n)

In [75]:
imagepath = str(-3)+'/'+str(-5)+'_'+str(7)+'/'

In [76]:
file = open(imagepath+str(n)+'.txt','a')
for i in range((len(gr)//3)+1):
    stri = ''
    for vec in gr[3*i:3*(i+1)]:
        stri += '['+str(vec[0])[:10]+','+str(vec[1])[:10]+'],'
    stri+='\n'
    file.write(stri)
file.close()


## Algebraic point groups

In [99]:
eltoforder_mod(6,19)

8

In [100]:
[pow(8,n,19) for n in range(7)]

[1, 8, 7, 18, 11, 12, 1]

In [50]:
def qr(a,p):
    r = pow(a,p//2,p)
    if r < 2:
        return r
    else:
        return -1

def eltoforder_mod(n,p):
    if (p-1)% n != 0:
        return 0
    x = 2
    while x < p:
        xe = pow(x,(p-1)//n,p)
        xens = {pow(xe,e,p) for e in range(n)}
        if len(xens)==n:
            return xe
        x+=1
    return 0

def compute_trace(fg:tuple[int],p:int)->int:
    f,g = fg
    return - sum([qr(x**3+f*x+g,p) for x in range(p)])

def get_ellcurves_in_class(ap):
    a,p = ap
    a = int(a)
    p = int(p)
    eqns = []
    ns = p-1
    d,c = discfac(a*a-4*p)
    while qr(ns,p)>=0:
        ns-=1
    for t in range(2,p):
        ft = (-3*t)%p
        gt = (2*t)%p
        tr = compute_trace((ft,gt),p)
        if tr == a:
            eqns.append((ft,gt))
        if tr == -a:
            ft = ns*ns*ft %p
            gt = ns*ns*ns*gt %p
            eqns.append((ft,gt))
    if p % 3 == 2 and a == 0:
        eqns+= [(0,1),(0,ns)]
    elif p % 3 == 1 and d == -3:
        gen6 = eltoforder_mod(6,p)
        cands = [(0,pow(gen6,e,p)) for e in range(6)]
        eqns+=[fg for fg in cands if compute_trace(fg,p)==a]
    if p % 4 == 3 and a == 0:
        eqns+=[(1,0),(ns,0)]
    elif p % 4 == 1 and d == -4:
        gen4 = eltoforder_mod(4,p)
        cands = [(pow(gen4,e,p),0) for e in range(4)]
        eqns+=[fg for fg in cands if compute_trace(fg,p)==a]
    return eqns

In [51]:
get_ellcurves_in_class((4,5))

[(2, 0)]

In [52]:
smallest_examples_twist

[(-4, 11),
 (-1, 5),
 (-3, 5),
 (-4, 5),
 (-5, 7),
 (-1, 11),
 (-6, 11),
 (-1, 17),
 (-1, 41)]

In [55]:
ecs_smex_tw = {ap:get_ellcurves_in_class(ap)[0] for ap in smallest_examples_twist}

In [56]:
ecs_smex_tw

{(-4, 11): (5, 7),
 (-1, 5): (2, 1),
 (-3, 5): (1, 1),
 (-4, 5): (3, 0),
 (-5, 7): (0, 3),
 (-1, 11): (3, 2),
 (-6, 11): (1, 3),
 (-1, 17): (8, 16),
 (-1, 41): (35, 29)}

In [60]:
def fg_to_str(fg,p=None):
    ec = 'y^2 = x^3 + '
    if fg[0]!=0:
        ec+=str(fg[0])+'x'
        if fg[1]!=0:
            ec+='+'
    if fg[1]!=0:
        ec+=str(fg[1])
    if p!= None:
        ec+=' mod '+str(p)
    return ec